# ANALYSIS AND RESULTS

In [1]:
from account import account_data
from report import read_reports
from pathlib import Path
import pandas as pd
import numpy as np
import os

In [2]:
#Root Path to the files in folders

account_folder = Path("bulk-reports/accounts")
report_folder = Path("bulk-reports/reports")

In [3]:
# calling the account data functoin to process the files and retrieve specific data
#output is a list of dictionaries
account_summary = account_data(account_folder)

#converting the list of dictionaries into a pandas Dataframe
account_df = pd.DataFrame(account_summary)
account_df.head()

Processing file: bulk-reports\accounts\0110c686-c6a0-4250-8dc5-832256a24f3d.json
Processed: 0110c686-c6a0-4250-8dc5-832256a24f3d.json
Processing file: bulk-reports\accounts\02fb16d1-1937-4e4e-b01c-624f0930f49f.json
Processed: 02fb16d1-1937-4e4e-b01c-624f0930f49f.json
Processing file: bulk-reports\accounts\04bb70b2-65d8-4b37-af87-507d58025153.json
Processed: 04bb70b2-65d8-4b37-af87-507d58025153.json
Processing file: bulk-reports\accounts\086abd29-f9e3-4bfd-b7de-4a5795f5a4b8.json
Processed: 086abd29-f9e3-4bfd-b7de-4a5795f5a4b8.json
Processing file: bulk-reports\accounts\0f331b0b-f1a2-4e5d-8284-33ca9bf41254.json
Processed: 0f331b0b-f1a2-4e5d-8284-33ca9bf41254.json
Processing file: bulk-reports\accounts\1bc25bc6-9a63-4ea7-ab83-fb764399da7b.json
Processed: 1bc25bc6-9a63-4ea7-ab83-fb764399da7b.json
Processing file: bulk-reports\accounts\1c609037-24c4-4d16-b0e6-db030039234d.json
Processed: 1c609037-24c4-4d16-b0e6-db030039234d.json
Processing file: bulk-reports\accounts\336b9f63-aa33-4a4c-886e

,uuid,employmentstatus,userID,bankname
0,0110c686-c6a0-4250-8dc5-832256a24f3d,FT_EMPLOYED,193700.0,ABSA
1,02fb16d1-1937-4e4e-b01c-624f0930f49f,FT_EMPLOYED,464600.0,CAPITEC
2,04bb70b2-65d8-4b37-af87-507d58025153,FT_EMPLOYED,253700.0,FIRST_NATIONAL_BANK
3,086abd29-f9e3-4bfd-b7de-4a5795f5a4b8,FT_EMPLOYED,500700.0,ABSA
4,0f331b0b-f1a2-4e5d-8284-33ca9bf41254,FT_EMPLOYED,337200.0,CAPITEC


In [ ]:
# calling the report data function to process the files and retrieve specific data
#output is a list of dictionaries
report_summary = read_reports(report_folder)

# Convert to DataFrame for analysis
report_df = pd.DataFrame(report_summary)
report_df.head()

In [5]:
# Using left join to merge account_df with report_df
merged_df = pd.merge(report_df, account_df, on="uuid", how="left")

In [7]:
# Convert credit_score column to integer
merged_df['credit_score'] = merged_df['credit_score'].astype(int)

### Average Credit Score

In [8]:
# Step 1: Average Credit Score
average_credit_score = merged_df['credit_score'].mean()

average_credit_score

104.38193791157103

### Number of users grouped by Employment status

In [9]:
unique_users_by_employment_status = merged_df.groupby('employmentstatus')['uuid'].nunique()

unique_users_by_employment_status

employmentstatus
FT_EMPLOYED      30
PT_EMPLOYED       2
SELF_EMPLOYED     3
STUDENT           1
UNEMPLOYED        2
WORK_AT_HOME      1
Name: uuid, dtype: int64

### Number of Users in Credit Score Ranges

In [10]:
latest_reports = merged_df.sort_values(['uuid', 'year', 'month'], ascending=[True, False, False]).drop_duplicates('uuid')

# Define bins for score ranges (0-50, 51-100, ...)
score_bins = np.arange(0, latest_reports['credit_score'].max() + 51, 50)
score_labels = [f"{int(b)}-{int(b + 49)}" for b in score_bins[:-1]]

# Create score ranges
latest_reports['score_range'] = pd.cut(latest_reports['credit_score'], bins=score_bins, labels=score_labels, right=False)

latest_reports[['uuid', 'credit_score', 'score_range']]

,uuid,credit_score,score_range
1418,0110c686-c6a0-4250-8dc5-832256a24f3d,577,550-599
1419,02fb16d1-1937-4e4e-b01c-624f0930f49f,595,550-599
1420,04bb70b2-65d8-4b37-af87-507d58025153,550,550-599
1421,086abd29-f9e3-4bfd-b7de-4a5795f5a4b8,0,0-49
1361,0f331b0b-f1a2-4e5d-8284-33ca9bf41254,644,600-649
1423,1bc25bc6-9a63-4ea7-ab83-fb764399da7b,564,550-599
1424,1c609037-24c4-4d16-b0e6-db030039234d,665,650-699
1425,336b9f63-aa33-4a4c-886e-3f7211ae9137,0,0-49
1426,3da8c835-1ee2-41df-8d92-60c8f638ea5a,670,650-699
1427,3dc87c55-fce3-4a36-b3fd-f96d79710cca,523,500-549


### Latest credit report data for each user

In [11]:
bank_data = latest_reports[['uuid', 'employmentstatus', 'bankname', 'active_bank_accounts', 'total_outstanding_balance']]

bank_data.head()

,uuid,employmentstatus,bankname,active_bank_accounts,total_outstanding_balance
1418,0110c686-c6a0-4250-8dc5-832256a24f3d,FT_EMPLOYED,ABSA,1,28464
1419,02fb16d1-1937-4e4e-b01c-624f0930f49f,FT_EMPLOYED,CAPITEC,0,0
1420,04bb70b2-65d8-4b37-af87-507d58025153,FT_EMPLOYED,FIRST_NATIONAL_BANK,0,0
1421,086abd29-f9e3-4bfd-b7de-4a5795f5a4b8,FT_EMPLOYED,ABSA,2,27659
1361,0f331b0b-f1a2-4e5d-8284-33ca9bf41254,FT_EMPLOYED,CAPITEC,1,2762


### Export the result as csv

In [12]:
output_folder = "output"
output_file = "bank_data.csv"

# Export the DataFrame to CSV
output_path = os.path.join(output_folder, output_file)
bank_data.to_csv(output_path, index=False)

print(f"CSV file saved successfully at: {output_path}")

CSV file saved successfully at: output\bank_data.csv
